# X2: High-Resolution Boundary Mapping (NLP)

**Paper-A (P0009) — X Series: Cross-Domain Generality**

| Item | Value |
|------|-------|
| Dataset | SST-2 (binary sentiment) |
| Model | DistilBERT (pretrained) |
| Noise | Symmetric, **η = 0.8** |
| Trusted ratio | 5% |
| λ grid | **{0.26, 0.28, 0.30, 0.32, 0.34, 0.36, 0.38, 0.40, 0.42, 0.44}** |
| Seeds | 0–19 (n=20 per λ) |
| Total runs | **200** |
| Epochs | 3 |
| Estimated time | ~2.5 hours (A100) |

**Purpose**: Map the X1b cliff (λ=0.30→0.40) at fine resolution.  
Identify variance peak and exact boundary location.


## 1. Setup

In [ ]:
!pip install -q transformers datasets accelerate

import os
import json
import time
import copy
import random
import warnings
import logging
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Subset

# Suppress noisy warnings from transformers model loading
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
logging.getLogger("transformers.configuration_utils").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", message=".*does not exist in your Colab secrets.*")
warnings.filterwarnings("ignore", message=".*unauthenticated requests.*")
os.environ["TRANSFORMERS_VERBOSITY"] = "error"

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


## 2. Configuration

In [ ]:
CONFIG = {
    "experiment": "exp_X2_boundary_mapping",
    "series": "X",
    "dataset": "SST-2",
    "num_classes": 2,
    "model_name": "distilbert-base-uncased",
    "noise_type": "symmetric",
    "noise_rate": 0.8,
    "trusted_ratio": 0.05,
    "lambda_values": [0.26, 0.28, 0.30, 0.32, 0.34, 0.36, 0.38, 0.40, 0.42, 0.44],
    "seeds": list(range(20)),
    "epochs": 3,
    "batch_size": 64,
    "learning_rate": 2e-5,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "max_seq_length": 128,
    "use_bf16": True,
    "max_train_samples": 16000,
    "csv_measure_every_n_steps": 25,
    "output_dir": "X2_results",
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)

n_used = min(int(67349 * 0.95), CONFIG["max_train_samples"])
steps_per_epoch = n_used // CONFIG["batch_size"]
total_steps = steps_per_epoch * CONFIG["epochs"]
n_runs = len(CONFIG["lambda_values"]) * len(CONFIG["seeds"])
est_per_run = total_steps * 25 / 1000 + 15

print(f"Total runs: {n_runs}")
print(f"λ values: {CONFIG['lambda_values']}")
print(f"Steps/epoch: ~{steps_per_epoch}, Total steps/run: ~{total_steps}")
print(f"Est. time/run: ~{est_per_run:.0f}s")
print(f"Est. total: ~{n_runs * est_per_run / 3600:.1f}h")


## 3. Google Drive Mount & Output Directory

In [ ]:
# ============================================================
# Google Drive Mount
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

# --- Save location on Drive ---
# Change this path if needed
DRIVE_BASE = "/content/drive/MyDrive/Paper-A_X_series"
DRIVE_OUTPUT = f"{DRIVE_BASE}/exp_X2_boundary_mapping"

os.makedirs(DRIVE_OUTPUT, exist_ok=True)
print(f"Drive output directory: {DRIVE_OUTPUT}")

# Also keep local copy for speed
LOCAL_OUTPUT = CONFIG["output_dir"]  # "X1_results"
os.makedirs(LOCAL_OUTPUT, exist_ok=True)

def save_to_drive(filename, data):
    """Save JSON to both local and Drive simultaneously."""
    # Local (fast)
    local_path = os.path.join(LOCAL_OUTPUT, filename)
    with open(local_path, "w") as f:
        json.dump(data, f, indent=2)
    # Drive (persistent)
    drive_path = os.path.join(DRIVE_OUTPUT, filename)
    with open(drive_path, "w") as f:
        json.dump(data, f, indent=2)
    return drive_path

def save_figure_to_drive(fig, filename, dpi=300):
    """Save figure to both local and Drive."""
    local_path = os.path.join(LOCAL_OUTPUT, filename)
    fig.savefig(local_path, dpi=dpi, bbox_inches="tight", facecolor="white")
    drive_path = os.path.join(DRIVE_OUTPUT, filename)
    fig.savefig(drive_path, dpi=dpi, bbox_inches="tight", facecolor="white")
    return drive_path

print("Save functions defined (local + Drive dual-write).")


## 4. Deterministic Seeding

In [ ]:
def set_seed(seed):
    """Set all random seeds for full reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)


## 5. Data Preparation

In [ ]:
# ============================================================
# Load SST-2 and tokenize
# ============================================================

tokenizer = DistilBertTokenizer.from_pretrained(CONFIG["model_name"])

print("Loading SST-2 dataset...")
raw_dataset = load_dataset("glue", "sst2")

# Use validation set as test (SST-2 test has no labels)
train_texts = list(raw_dataset["train"]["sentence"])
train_labels = list(raw_dataset["train"]["label"])
test_texts = list(raw_dataset["validation"]["sentence"])
test_labels = list(raw_dataset["validation"]["label"])

print(f"Train: {len(train_texts)} samples")
print(f"Test:  {len(test_texts)} samples")
print(f"Label distribution (train): {np.bincount(train_labels)}")

# Tokenize
print("Tokenizing...")

def tokenize_batch(texts, max_length):
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

train_encodings = tokenize_batch(train_texts, CONFIG["max_seq_length"])
test_encodings = tokenize_batch(test_texts, CONFIG["max_seq_length"])

train_labels_tensor = torch.tensor(train_labels)
test_labels_tensor = torch.tensor(test_labels)

print("Tokenization complete.")


In [ ]:
# ============================================================
# Custom Dataset
# ============================================================

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }
        return item

test_dataset = TextDataset(test_encodings, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Test dataset: {len(test_dataset)} samples")


## 6. Noise Injection & Trusted Subset Separation

In [ ]:
def prepare_noisy_and_trusted(train_labels, noise_rate, trusted_ratio,
                              num_classes, seed, max_noisy_samples=None):
    """
    1) Separate trusted subset (clean) BEFORE noise injection
    2) Inject symmetric noise into remaining data
    3) Optionally subsample the noisy pool for speed
    Returns: noisy_labels, trusted_indices, noisy_indices
    """
    rng = np.random.RandomState(seed + 10000)
    n = len(train_labels)
    labels = np.array(train_labels)
    indices = np.arange(n)

    # --- Step 1: Stratified trusted subset ---
    trusted_indices = []
    for c in range(num_classes):
        class_idx = indices[labels == c]
        n_class_trusted = max(1, int(len(class_idx) * trusted_ratio))
        selected = rng.choice(class_idx, size=n_class_trusted, replace=False)
        trusted_indices.extend(selected)
    trusted_indices = np.array(sorted(trusted_indices))

    # Remaining = noisy pool
    noisy_mask = np.ones(n, dtype=bool)
    noisy_mask[trusted_indices] = False
    noisy_indices = indices[noisy_mask]

    # --- Step 2: Inject symmetric noise ---
    noisy_labels = labels.copy()
    n_noisy = len(noisy_indices)
    n_flip = int(n_noisy * noise_rate)
    flip_idx = rng.choice(noisy_indices, size=n_flip, replace=False)
    for idx in flip_idx:
        original = noisy_labels[idx]
        candidates = [c for c in range(num_classes) if c != original]
        noisy_labels[idx] = rng.choice(candidates)

    # --- Step 3: Subsample noisy pool (if requested) ---
    if max_noisy_samples and len(noisy_indices) > max_noisy_samples:
        noisy_indices = rng.choice(noisy_indices, size=max_noisy_samples, replace=False)
        noisy_indices = np.sort(noisy_indices)

    actual_noise = np.mean(noisy_labels[noisy_indices] != labels[noisy_indices])
    print(f"  Trusted: {len(trusted_indices)} ({len(trusted_indices)/n*100:.1f}%) | "
          f"Noisy: {len(noisy_indices)} (η_actual={actual_noise:.3f})")

    return torch.tensor(noisy_labels), trusted_indices, noisy_indices

# Quick test
print("Testing noise injection + subsampling (seed=0)...")
_, ti, ni = prepare_noisy_and_trusted(train_labels, 0.4, 0.05, 2, seed=0,
                                       max_noisy_samples=16000)
print(f"  → {len(ti)} trusted + {len(ni)} noisy\n")


## 7. Dual-Gradient Training Core

In [ ]:
def get_head_param_names(model):
    """Return parameter names of the classification head."""
    return [n for n, _ in model.named_parameters()
            if "classifier" in n or "pre_classifier" in n]


def extract_grads(model):
    """Extract current gradients as a flat dict."""
    return {n: p.grad.clone() for n, p in model.named_parameters()
            if p.grad is not None}


def normalize_grad_dict(grads):
    """L2-normalize gradient vector (across all params)."""
    flat = torch.cat([g.flatten() for g in grads.values()])
    norm = flat.norm()
    if norm > 0:
        return {n: g / norm for n, g in grads.items()}, norm.item()
    return grads, 0.0


def cosine_sim(grads_a, grads_b, param_names=None):
    """Cosine similarity between two gradient dicts."""
    if param_names:
        keys = [n for n in param_names if n in grads_a and n in grads_b]
    else:
        keys = sorted(set(grads_a) & set(grads_b))
    if not keys:
        return 0.0
    va = torch.cat([grads_a[n].flatten() for n in keys])
    vb = torch.cat([grads_b[n].flatten() for n in keys])
    return F.cosine_similarity(va.unsqueeze(0), vb.unsqueeze(0)).item()


def set_mixed_grad(model, g_struct_norm, g_value_norm, lam):
    """Set param.grad = (1-λ)ĝ_struct + λĝ_value for optimizer.step()."""
    for n, p in model.named_parameters():
        if n in g_struct_norm and n in g_value_norm:
            p.grad = (1 - lam) * g_struct_norm[n] + lam * g_value_norm[n]
        elif n in g_struct_norm:
            p.grad = (1 - lam) * g_struct_norm[n]
        elif n in g_value_norm:
            p.grad = lam * g_value_norm[n]

print("Dual-gradient core functions defined.")


In [ ]:
@torch.no_grad()
def evaluate(model, test_loader, device):
    """Evaluate model on test set."""
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0
    n_batches = 0

    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        preds = outputs.logits.argmax(dim=-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += outputs.loss.item()
        n_batches += 1

    acc = correct / total
    error = 1.0 - acc
    avg_loss = total_loss / n_batches
    model.train()
    return acc, error, avg_loss

print("Evaluation function defined.")


## 8. Single Run Execution

In [ ]:
def run_single_experiment(lam, seed, config):
    """
    Execute one dual-gradient training run.
    - bf16 mixed precision for speed
    - Batch-level updates with AdamW + warmup scheduler
    - Subsampled noisy pool for feasible runtime
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    set_seed(seed)
    t_start = time.time()

    # --- Prepare data ---
    noisy_labels, trusted_idx, noisy_idx = prepare_noisy_and_trusted(
        train_labels, config["noise_rate"], config["trusted_ratio"],
        config["num_classes"], seed,
        max_noisy_samples=config.get("max_train_samples")
    )

    noisy_dataset = TextDataset(
        {k: v[noisy_idx] for k, v in train_encodings.items()},
        noisy_labels[noisy_idx]
    )
    trusted_dataset = TextDataset(
        {k: v[trusted_idx] for k, v in train_encodings.items()},
        noisy_labels[trusted_idx]
    )

    struct_loader = DataLoader(noisy_dataset, batch_size=config["batch_size"],
                               shuffle=True, drop_last=True)
    value_loader = DataLoader(trusted_dataset,
                              batch_size=min(config["batch_size"], len(trusted_dataset)),
                              shuffle=True, drop_last=False)

    # --- Initialize model ---
    set_seed(seed)
    model = DistilBertForSequenceClassification.from_pretrained(
        config["model_name"], num_labels=config["num_classes"]
    ).to(device)
    model.train()

    head_params = get_head_param_names(model)

    # --- Optimizer + Scheduler ---
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config["learning_rate"],
        weight_decay=config["weight_decay"]
    )

    steps_per_epoch = len(struct_loader)
    total_steps = steps_per_epoch * config["epochs"]
    warmup_steps = int(total_steps * config["warmup_ratio"])

    def lr_lambda(step):
        if step < warmup_steps:
            return step / max(1, warmup_steps)
        return max(0.0, (total_steps - step) / max(1, total_steps - warmup_steps))

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    # --- Mixed precision ---
    use_amp = config.get("use_bf16", False) and device.type == "cuda"
    amp_dtype = torch.bfloat16 if use_amp else torch.float32

    # --- Training loop ---
    epoch_logs = []
    all_csv = []
    measure_every = config.get("csv_measure_every_n_steps", 25)
    global_step = 0

    for epoch in range(config["epochs"]):
        epoch_loss_s = 0.0
        epoch_loss_v = 0.0
        epoch_csv = []
        n_steps = 0
        value_iter = iter(value_loader)

        for struct_batch in struct_loader:
            # === Struct gradient ===
            optimizer.zero_grad()
            s_ids = struct_batch["input_ids"].to(device)
            s_mask = struct_batch["attention_mask"].to(device)
            s_labels = struct_batch["labels"].to(device)

            with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                out_s = model(input_ids=s_ids, attention_mask=s_mask, labels=s_labels)
            out_s.loss.backward()
            g_struct = extract_grads(model)
            loss_s = out_s.loss.item()

            # === Value gradient ===
            optimizer.zero_grad()
            try:
                value_batch = next(value_iter)
            except StopIteration:
                value_iter = iter(value_loader)
                value_batch = next(value_iter)

            v_ids = value_batch["input_ids"].to(device)
            v_mask = value_batch["attention_mask"].to(device)
            v_labels = value_batch["labels"].to(device)

            with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                out_v = model(input_ids=v_ids, attention_mask=v_mask, labels=v_labels)
            out_v.loss.backward()
            g_value = extract_grads(model)
            loss_v = out_v.loss.item()

            # === Normalize & Mix ===
            g_struct_n, _ = normalize_grad_dict(g_struct)
            g_value_n, _ = normalize_grad_dict(g_value)
            set_mixed_grad(model, g_struct_n, g_value_n, lam)

            # === Step ===
            optimizer.step()
            scheduler.step()
            global_step += 1
            n_steps += 1

            epoch_loss_s += loss_s
            epoch_loss_v += loss_v

            # === c_sv (periodic) ===
            if global_step % measure_every == 0:
                c = cosine_sim(g_struct, g_value, head_params)
                epoch_csv.append(c)
                all_csv.append(c)

        # --- Evaluate ---
        test_acc, test_error, test_loss = evaluate(model, test_loader, device)
        mean_csv = float(np.mean(epoch_csv)) if epoch_csv else 0.0

        epoch_log = {
            "epoch": epoch + 1,
            "steps": n_steps,
            "test_acc": round(test_acc, 4),
            "test_error": round(test_error, 4),
            "test_loss": round(test_loss, 4),
            "loss_struct": round(epoch_loss_s / max(n_steps, 1), 4),
            "loss_value": round(epoch_loss_v / max(n_steps, 1), 4),
            "c_sv_head": round(mean_csv, 4),
        }
        epoch_logs.append(epoch_log)

        print(f"    Ep {epoch+1}/{config['epochs']} ({n_steps} steps): "
              f"err={test_error:.4f}  acc={test_acc:.4f}  "
              f"c_sv={mean_csv:.4f}")

    # --- Final ---
    final_acc, final_error, _ = evaluate(model, test_loader, device)
    elapsed = time.time() - t_start
    mean_csv_all = float(np.mean(all_csv)) if all_csv else 0.0

    result = {
        "experiment_id": f"X2-{seed:03d}-lam{lam:.2f}",
        "experiment": config["experiment"],
        "dataset": config["dataset"],
        "model": config["model_name"],
        "noise_type": config["noise_type"],
        "noise_rate": config["noise_rate"],
        "trusted_ratio": config["trusted_ratio"],
        "lambda": lam,
        "seed": seed,
        "test_acc": round(final_acc, 4),
        "test_error": round(final_error, 4),
        "best_acc": round(max(log["test_acc"] for log in epoch_logs), 4),
        "best_error": round(min(log["test_error"] for log in epoch_logs), 4),
        "avg_cos_struct_value": round(mean_csv_all, 4),
        "total_steps": global_step,
        "epochs": config["epochs"],
        "epoch_logs": epoch_logs,
        "time_seconds": round(elapsed, 1),
    }

    del model, optimizer, scheduler
    torch.cuda.empty_cache()
    return result

print("Single run function defined (bf16 + subsampled + batch-level).")


## 9. Main Execution Loop

In [ ]:
# ============================================================
# Main Loop: All λ × All Seeds
# ============================================================

all_results = []
total_runs = len(CONFIG["lambda_values"]) * len(CONFIG["seeds"])
run_idx = 0

print("=" * 70)
print(f"X1: Coarse λ Sweep — {total_runs} runs")
print(f"  λ values: {CONFIG['lambda_values']}")
print(f"  Seeds: {CONFIG['seeds']}")
print(f"  η = {CONFIG['noise_rate']}, trusted = {CONFIG['trusted_ratio']}")
print("=" * 70)

t_total_start = time.time()

for lam in CONFIG["lambda_values"]:
    print(f"\n{'='*60}")
    print(f"  λ = {lam:.2f}")
    print(f"{'='*60}")

    lambda_results = []

    for seed in CONFIG["seeds"]:
        run_idx += 1
        print(f"\n  --- Run {run_idx}/{total_runs}: λ={lam:.2f}, seed={seed} ---")

        result = run_single_experiment(lam, seed, CONFIG)
        lambda_results.append(result)
        all_results.append(result)

        # Progress
        elapsed_total = time.time() - t_total_start
        avg_per_run = elapsed_total / run_idx
        remaining = avg_per_run * (total_runs - run_idx)
        print(f"    → err={result['test_error']:.4f}  c_sv={result['avg_cos_struct_value']:.4f}  "
              f"time={result['time_seconds']:.0f}s  "
              f"[{run_idx}/{total_runs}, ETA: {remaining/60:.0f} min]")

    # Summary for this λ
    errors = [r["test_error"] for r in lambda_results]
    cos_values = [r["avg_cos_struct_value"] for r in lambda_results]
    print(f"\n  λ={lam:.2f} SUMMARY:")
    print(f"    error: mean={np.mean(errors):.4f} ± {np.std(errors):.4f}")
    print(f"    error: min={np.min(errors):.4f}  max={np.max(errors):.4f}")
    print(f"    c_sv:  mean={np.mean(cos_values):.4f} ± {np.std(cos_values):.4f}")

    # Save intermediate results for this λ (local + Drive)
    fname = f"X2_lambda{lam:.2f}_results.json"
    drive_path = save_to_drive(fname, lambda_results)
    print(f"    Saved: {fname} → Drive ✓")

# ============================================================
# Save all results
# ============================================================
t_total = time.time() - t_total_start

save_to_drive("X2_results.json", all_results)
save_to_drive("X2_config.json", CONFIG)

print(f"\n{'='*70}")
print(f"COMPLETE: {total_runs} runs in {t_total/60:.1f} minutes ({t_total/3600:.1f} hours)")
print(f"Results saved to Drive: {DRIVE_OUTPUT}")
print(f"{'='*70}")


## 10. Analysis & Visualization

In [ ]:
import matplotlib.pyplot as plt

# Load results (prefer local, fallback to Drive)
local_path = os.path.join(CONFIG["output_dir"], "X2_results.json")
drive_path = os.path.join(DRIVE_OUTPUT, "X2_results.json")

if os.path.exists(local_path):
    with open(local_path) as f:
        results = json.load(f)
    print(f"Loaded from local: {local_path}")
elif os.path.exists(drive_path):
    with open(drive_path) as f:
        results = json.load(f)
    print(f"Loaded from Drive: {drive_path}")
else:
    raise FileNotFoundError("No results found! Run main loop first.")

# Organize by λ
lambda_data = {}
for r in results:
    lam = r["lambda"]
    if lam not in lambda_data:
        lambda_data[lam] = {"errors": [], "cos_values": [], "accs": []}
    lambda_data[lam]["errors"].append(r["test_error"])
    lambda_data[lam]["cos_values"].append(r["avg_cos_struct_value"])
    lambda_data[lam]["accs"].append(r["test_acc"])

lambdas = sorted(lambda_data.keys())

# ============================================================
# Figure 1: Phase Structure Overview (3 panels)
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Panel (a): Error distribution by λ
ax = axes[0]
positions = range(len(lambdas))
bp = ax.boxplot(
    [lambda_data[l]["errors"] for l in lambdas],
    positions=positions, widths=0.6, patch_artist=True
)
for patch in bp["boxes"]:
    patch.set_facecolor("#1F77B4")
    patch.set_alpha(0.5)

# Overlay individual points
for i, lam in enumerate(lambdas):
    jitter = np.random.normal(0, 0.05, len(lambda_data[lam]["errors"]))
    ax.scatter(
        [i + j for j in jitter],
        lambda_data[lam]["errors"],
        color="#D62728", alpha=0.6, s=20, zorder=3
    )

ax.set_xticks(positions)
ax.set_xticklabels([f"{l:.2f}" for l in lambdas])
ax.set_xlabel("λ", fontsize=14)
ax.set_ylabel("Test Error", fontsize=14)
ax.set_title("(a) Test Error Distribution by λ", fontsize=13, fontweight="bold")
ax.grid(True, alpha=0.3)

# Panel (b): Variance by λ
ax = axes[1]
variances = [np.var(lambda_data[l]["errors"]) for l in lambdas]
ax.bar(positions, variances, color="#FF7F0E", edgecolor="black")
for i, v in enumerate(variances):
    ax.text(i, v + max(variances)*0.02, f"{v:.4f}", ha="center", fontsize=9)
ax.set_xticks(positions)
ax.set_xticklabels([f"{l:.2f}" for l in lambdas])
ax.set_xlabel("λ", fontsize=14)
ax.set_ylabel("Variance of Test Error", fontsize=14)
ax.set_title("(b) Run-to-Run Variance (peak = critical region)", fontsize=13, fontweight="bold")
ax.grid(True, alpha=0.3)

# Panel (c): c_sv distribution by λ
ax = axes[2]
bp2 = ax.boxplot(
    [lambda_data[l]["cos_values"] for l in lambdas],
    positions=positions, widths=0.6, patch_artist=True
)
for patch in bp2["boxes"]:
    patch.set_facecolor("#2CA02C")
    patch.set_alpha(0.5)

for i, lam in enumerate(lambdas):
    jitter = np.random.normal(0, 0.05, len(lambda_data[lam]["cos_values"]))
    ax.scatter(
        [i + j for j in jitter],
        lambda_data[lam]["cos_values"],
        color="#9467BD", alpha=0.6, s=20, zorder=3
    )

ax.set_xticks(positions)
ax.set_xticklabels([f"{l:.2f}" for l in lambdas])
ax.set_xlabel("λ", fontsize=14)
ax.set_ylabel("c_sv (cos similarity, head only)", fontsize=14)
ax.set_title("(c) Gradient Geometry by λ", fontsize=13, fontweight="bold")
ax.grid(True, alpha=0.3)

fig.suptitle("X1: Phase Structure in NLP (SST-2, DistilBERT, η=0.4)",
             fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
save_figure_to_drive(fig, "X2_boundary_mapping.png")
plt.show()
print(f"Saved: X2_boundary_mapping.png → Drive ✓")


## 11. Boundary Analysis

In [ ]:
from scipy import stats

print("=" * 60)
print("X2: Boundary Analysis")
print("=" * 60)

# Compute derivative (slope between adjacent λ)
lambdas_sorted = sorted(lambda_data.keys())
slopes = []
for i in range(len(lambdas_sorted)-1):
    l1, l2 = lambdas_sorted[i], lambdas_sorted[i+1]
    m1 = np.mean(lambda_data[l1])
    m2 = np.mean(lambda_data[l2])
    slope = (m2 - m1) / (l2 - l1)
    slopes.append((l1, l2, slope))
    print(f"  λ={l1:.2f}→{l2:.2f}: slope = {slope:.2f} (Δerr = {m2-m1:.4f})")

# Find steepest
steepest = min(slopes, key=lambda x: x[2])
print(f"\nSteepest transition: λ={steepest[0]:.2f}→{steepest[1]:.2f} (slope={steepest[2]:.2f})")

# Variance analysis
print("\nVariance by λ:")
variances = {}
for lam in lambdas_sorted:
    v = np.var(lambda_data[lam])
    variances[lam] = v
    print(f"  λ={lam:.2f}: var={v:.6f} ({v*10000:.2f} ×10⁴)")

peak_lam = max(variances, key=variances.get)
print(f"\nPeak variance at λ={peak_lam:.2f}")

# Separation tests
print("\nAdjacent λ separation (t-test):")
for i in range(len(lambdas_sorted)-1):
    l1, l2 = lambdas_sorted[i], lambdas_sorted[i+1]
    e1 = lambda_data[l1]
    e2 = lambda_data[l2]
    t, p = stats.ttest_ind(e1, e2)
    overlap = "NONE" if min(e1) > max(e2) or min(e2) > max(e1) else "partial"
    print(f"  λ={l1:.2f} vs {l2:.2f}: t={t:.2f}, p={p:.2e}, overlap={overlap}")


## 12. Summary Table

In [ ]:
# ============================================================
# Final Summary Table
# ============================================================

print(f"\n{'='*80}")
print(f"{'X2 RESULTS SUMMARY':^80}")
print(f"{'='*80}")
print(f"{'λ':>6} | {'mean_err':>10} | {'std_err':>10} | {'min_err':>10} | {'max_err':>10} | {'variance':>10} | {'mean_c_sv':>10}")
print("-" * 80)

for lam in lambdas:
    errors = lambda_data[lam]["errors"]
    cos_vals = lambda_data[lam]["cos_values"]
    print(f"{lam:>6.2f} | {np.mean(errors):>10.4f} | {np.std(errors):>10.4f} | "
          f"{np.min(errors):>10.4f} | {np.max(errors):>10.4f} | "
          f"{np.var(errors):>10.6f} | {np.mean(cos_vals):>10.4f}")

print("-" * 80)
print(f"Total runs: {len(results)}")
print(f"Total time: {sum(r['time_seconds'] for r in results)/60:.1f} min")

# Identify critical region
peak_var_lambda = lambdas[np.argmax(variances)]
print(f"\nPeak variance at λ = {peak_var_lambda:.2f}")
print(f"  → Suggested critical region for X2: λ ∈ [{peak_var_lambda-0.04:.2f}, {peak_var_lambda+0.06:.2f}]")
print(f"  → Suggested λ for X4 (metastability): {peak_var_lambda+0.04:.2f}")


## 13. Download Results

In [ ]:
# Results are already saved to Google Drive
print(f"All results saved to: {DRIVE_OUTPUT}")
print(f"Contents:")
for f in sorted(os.listdir(DRIVE_OUTPUT)):
    size = os.path.getsize(os.path.join(DRIVE_OUTPUT, f))
    print(f"  {f}  ({size/1024:.1f} KB)")

# Optional: also zip for local download
import shutil
shutil.make_archive("X1_results", "zip", LOCAL_OUTPUT)
print(f"\nLocal zip also available: X1_results.zip")
try:
    from google.colab import files
    files.download("X1_results.zip")
except:
    pass
